In [1]:
%matplotlib inline
# Enabling reloading extension to use external python files
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os
from datetime import date, datetime,timedelta
import math
import columns
import datasets
import seasons
import directory_utils

In [3]:
# Setup matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')

In [4]:
data_frame = pd.read_csv(datasets.FIRST_CLEAN_DATA_SET_111_PATH, encoding=datasets.DEFAULT_CSV_ENCODING)

/home/felipepedroso/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data_frame.shape

(994234, 19)

In [6]:
data_frame.columns

Index(['Cod', 'Ano', 'Dia Juliano', 'Hora - minuto',
       'Veloc. Instantânea do Vento a 5 metros (m/s)',
       'Veloc. Instantânea do Vento a 2 metros (m/s)',
       'Direção do Vento no instante da aquisição (°)',
       'Radiação Incidente total de 10 leituras (KW/m2)',
       'Radiação Refletida Total de 10 leituras (KW/m2)',
       'Fluxo Positivo de calor no solo total de 10 leituras (W/m2)',
       'Fluxo negativo de calor no solo . total de 10 leituras (W/m2)',
       'Umidade relativa Média (%)', 'Temperatura do Ar Média (°C)',
       'Temperatura do solo  1 Média (°C) à 3cm no solo',
       'Temperatura do solo 2Média (°C) à 6cm no solo ',
       'Temperatura do solo 3Média (°C) Colocado sensor de UV em 31/07/09',
       'Pressão (hPa)', 'Chuva total do período (mm)', 'Data'],
      dtype='object')

### Extracting daily information

In [7]:
daily_analysis_columns = [columns.TEMPERATURE,columns.HUMIDITY,columns.PRESSURE]

In [8]:
daily_average_from_columns = data_frame.groupby(columns.DATE)[daily_analysis_columns].mean()
daily_average_from_columns.head()

,Temperatura do Ar Média (°C),Umidade relativa Média (%),Pressão (hPa)
Data,,,
1998-01-01,27.020000,90.190417,947.798611
1998-01-02,28.737986,83.891111,948.722222
1998-01-03,26.078472,90.326389,945.666667
1998-01-04,24.909861,93.515486,943.770833
1998-01-05,27.336389,87.693681,943.201389


In [9]:
daily_max_from_columns = data_frame.groupby(columns.DATE)[daily_analysis_columns].max()
new_column_names = [columns.MAX_TEMP,columns.MAX_HUMIDITY,columns.MAX_PRESSURE]
daily_max_from_columns.columns = new_column_names
daily_max_from_columns.head()

,Temperatura do Ar Máxima (°C),Umidade relativa Máxima (%),Pressão Máxima (hPa)
Data,,,
1998-01-01,34.88,100.0,962.0
1998-01-02,33.95,100.0,967.0
1998-01-03,33.55,100.0,957.0
1998-01-04,32.49,100.0,948.0
1998-01-05,34.31,100.0,956.0


In [10]:
daily_min_from_columns = data_frame.groupby(columns.DATE)[daily_analysis_columns].min()
new_column_names = [columns.MIN_TEMP,columns.MIN_HUMIDITY,columns.MIN_PRESSURE]
daily_min_from_columns.columns = new_column_names
daily_min_from_columns.head()

,Temperatura do Ar Mínima (°C),Umidade relativa Mínima (%),Pressão Mínima (hPa)
Data,,,
1998-01-01,20.51,60.06,942.0
1998-01-02,22.20,54.20,942.0
1998-01-03,20.41,70.30,941.0
1998-01-04,20.31,69.33,940.0
1998-01-05,21.18,59.10,938.0


In [11]:
daily_analysis_data_frame = pd.merge(daily_max_from_columns,daily_min_from_columns, on=columns.DATE)
daily_analysis_data_frame = pd.merge(daily_analysis_data_frame, daily_average_from_columns, on=columns.DATE)
daily_analysis_data_frame.head()

,Temperatura do Ar Máxima (°C),Umidade relativa Máxima (%),Pressão Máxima (hPa),Temperatura do Ar Mínima (°C),Umidade relativa Mínima (%),Pressão Mínima (hPa),Temperatura do Ar Média (°C),Umidade relativa Média (%),Pressão (hPa)
Data,,,,,,,,,
1998-01-01,34.88,100.0,962.0,20.51,60.06,942.0,27.020000,90.190417,947.798611
1998-01-02,33.95,100.0,967.0,22.20,54.20,942.0,28.737986,83.891111,948.722222
1998-01-03,33.55,100.0,957.0,20.41,70.30,941.0,26.078472,90.326389,945.666667
1998-01-04,32.49,100.0,948.0,20.31,69.33,940.0,24.909861,93.515486,943.770833
1998-01-05,34.31,100.0,956.0,21.18,59.10,938.0,27.336389,87.693681,943.201389


In [12]:
daily_analysis_data_frame[columns.VARIATION_TEMP] = daily_analysis_data_frame[columns.MAX_TEMP]-daily_analysis_data_frame[columns.MIN_TEMP]
daily_analysis_data_frame[columns.VARIATION_HUMIDITY] = daily_analysis_data_frame[columns.MAX_HUMIDITY]-daily_analysis_data_frame[columns.MIN_HUMIDITY]
daily_analysis_data_frame[columns.VARIATION_PRESSURE] = daily_analysis_data_frame[columns.MAX_PRESSURE]-daily_analysis_data_frame[columns.MIN_PRESSURE]
daily_analysis_data_frame.head()

,Temperatura do Ar Máxima (°C),Umidade relativa Máxima (%),Pressão Máxima (hPa),Temperatura do Ar Mínima (°C),Umidade relativa Mínima (%),Pressão Mínima (hPa),Temperatura do Ar Média (°C),Umidade relativa Média (%),Pressão (hPa),Variação da Temperatura do Ar (°C),Variação da Umidade relativa (%),Variação da Pressão (hPa)
Data,,,,,,,,,,,,
1998-01-01,34.88,100.0,962.0,20.51,60.06,942.0,27.020000,90.190417,947.798611,14.37,39.94,20.0
1998-01-02,33.95,100.0,967.0,22.20,54.20,942.0,28.737986,83.891111,948.722222,11.75,45.80,25.0
1998-01-03,33.55,100.0,957.0,20.41,70.30,941.0,26.078472,90.326389,945.666667,13.14,29.70,16.0
1998-01-04,32.49,100.0,948.0,20.31,69.33,940.0,24.909861,93.515486,943.770833,12.18,30.67,8.0
1998-01-05,34.31,100.0,956.0,21.18,59.10,938.0,27.336389,87.693681,943.201389,13.13,40.90,18.0


In [13]:
daily_analysis_data_frame.index = pd.to_datetime(daily_analysis_data_frame.index, format='%Y-%m-%d', utc=True)
daily_analysis_data_frame[columns.DAY_OF_YEAR] = daily_analysis_data_frame.index.map(lambda date: date.timetuple().tm_yday)
daily_analysis_data_frame[columns.YEAR] = daily_analysis_data_frame.index.map(lambda date: date.year)
daily_analysis_data_frame[columns.MONTH] = daily_analysis_data_frame.index.map(lambda date: date.month)
daily_analysis_data_frame[columns.DAY] = daily_analysis_data_frame.index.map(lambda date: date.day)
daily_analysis_data_frame[columns.SEASON] = daily_analysis_data_frame.index.map(lambda date: seasons.get_season(date))

In [14]:
min_amount_days_window = 15
window_delta_amount = math.ceil((min_amount_days_window / 2)-1)
window_delta = timedelta(days=window_delta_amount)
quantile_value = 0.9

def get_quantile(index, column):
    begin_date = index - window_delta
    end_date = index + window_delta
    data_window = daily_analysis_data_frame.loc[begin_date:end_date]
    return data_window[column].quantile(quantile_value)

daily_analysis_data_frame[columns.QUANTILE_MAX_TEMP_FIFTEEN_DAYS] = daily_analysis_data_frame.index.map(lambda index: get_quantile(index, columns.MAX_TEMP))

daily_analysis_data_frame[columns.QUANTILE_MIN_TEMP_FIFTEEN_DAYS] = daily_analysis_data_frame.index.map(lambda index: get_quantile(index,columns.MIN_TEMP))

daily_analysis_data_frame.head()

,Temperatura do Ar Máxima (°C),Umidade relativa Máxima (%),Pressão Máxima (hPa),Temperatura do Ar Mínima (°C),Umidade relativa Mínima (%),Pressão Mínima (hPa),Temperatura do Ar Média (°C),Umidade relativa Média (%),Pressão (hPa),Variação da Temperatura do Ar (°C),Variação da Umidade relativa (%),Variação da Pressão (hPa),Dia Juliano,Ano,Mês,Dia,Estação Metereológica do Ano,Percentil Temperatura Max (15 dias),Percentil Temperatura Min (15 dias)
Data,,,,,,,,,,,,,,,,,,,
1998-01-01 00:00:00+00:00,34.88,100.0,962.0,20.51,60.06,942.0,27.020000,90.190417,947.798611,14.37,39.94,20.0,1,1998,1,1,Verão,35.378,21.486
1998-01-02 00:00:00+00:00,33.95,100.0,967.0,22.20,54.20,942.0,28.737986,83.891111,948.722222,11.75,45.80,25.0,2,1998,1,2,Verão,35.212,21.384
1998-01-03 00:00:00+00:00,33.55,100.0,957.0,20.41,70.30,941.0,26.078472,90.326389,945.666667,13.14,29.70,16.0,3,1998,1,3,Verão,35.046,21.282
1998-01-04 00:00:00+00:00,32.49,100.0,948.0,20.31,69.33,940.0,24.909861,93.515486,943.770833,12.18,30.67,8.0,4,1998,1,4,Verão,34.880,21.180
1998-01-05 00:00:00+00:00,34.31,100.0,956.0,21.18,59.10,938.0,27.336389,87.693681,943.201389,13.13,40.90,18.0,5,1998,1,5,Verão,34.823,21.165


In [15]:
# Reordering columns
_ORDER = [
    columns.YEAR,
    columns.MONTH,
    columns.DAY,
    columns.DAY_OF_YEAR,
    columns.SEASON,
    columns.TEMPERATURE, 
    columns.MAX_TEMP, 
    columns.MIN_TEMP, 
    columns.VARIATION_TEMP,
    columns.QUANTILE_MAX_TEMP_FIFTEEN_DAYS,
    columns.QUANTILE_MIN_TEMP_FIFTEEN_DAYS,
    columns.HUMIDITY, 
    columns.MAX_HUMIDITY,
    columns.MIN_HUMIDITY,
    columns.VARIATION_HUMIDITY,
    columns.PRESSURE,
    columns.MAX_PRESSURE,
    columns.MIN_PRESSURE,
    columns.VARIATION_PRESSURE
]
daily_analysis_data_frame = daily_analysis_data_frame[_ORDER]

In [16]:
directory_utils.create_if_doesnt_exist(datasets.DAILY_ANALYSIS_DIRECTORY_PATH):
daily_analysis_data_frame.to_csv(datasets.DAILY_ANALYSIS_DATA_SET_111_PATH , index = True, encoding=datasets.DEFAULT_CSV_ENCODING)

SyntaxError: invalid syntax (<ipython-input-16-06f23f4b3b87>, line 1)

In [ ]:
daily_analysis_data_frame.head()

In [ ]:
min_x_value = daily_analysis_data_frame[columns.VARIATION_TEMP].min() - 5
max_x_value = daily_analysis_data_frame[columns.VARIATION_TEMP].max() + 5

min_y_value = daily_analysis_data_frame[columns.VARIATION_HUMIDITY].min() - 5
max_y_value = daily_analysis_data_frame[columns.VARIATION_HUMIDITY].max() + 5

for i in daily_analysis_data_frame[columns.YEAR].unique():
    data_2plot = daily_analysis_data_frame[(daily_analysis_data_frame[columns.YEAR] == i)]

    plt.figure(figsize=(13,4))
    plt.title(str(i))    
    plt.scatter(data_2plot[columns.VARIATION_TEMP], data_2plot[columns.VARIATION_HUMIDITY])
    plt.xlabel(columns.VARIATION_TEMP)
    plt.ylabel(columns.VARIATION_HUMIDITY)
    plt.grid(True, linestyle='--', alpha = 0.5)
    plt.xlim(min_x_value, max_x_value)
    plt.ylim(min_y_value, max_y_value)
    plt.show()

In [ ]:
daily_analysis_data_frame[[columns.VARIATION_TEMP,columns.VARIATION_HUMIDITY]].corr()


In [ ]:
min_x_value = daily_analysis_data_frame[columns.MAX_TEMP].min() - 5
max_x_value = daily_analysis_data_frame[columns.MAX_TEMP].max() + 5

min_y_value = daily_analysis_data_frame[columns.DAY_OF_YEAR].min() - 5
max_y_value = daily_analysis_data_frame[columns.DAY_OF_YEAR].max() + 5

for i in daily_analysis_data_frame[columns.YEAR].unique():
    data_2plot = daily_analysis_data_frame[(daily_analysis_data_frame[columns.YEAR] == i)]
#     data_2plot = data_2plot[data_2plot[_DAY_OF_YEAR_COLUMN]>=200]
#     data_2plot = data_2plot[data_2plot[_DAY_OF_YEAR_COLUMN]<=250]
    
    plt.figure(figsize=(20,10))
    plt.title(str(i))    
    plt.plot( columns.DAY_OF_YEAR, columns.MAX_TEMP, data=data_2plot, color='red', linewidth=1,marker='o', markerfacecolor='red', markersize=4)
    plt.plot( columns.DAY_OF_YEAR, columns.MIN_TEMP, data=data_2plot, color='skyblue', linewidth=1,marker='o', markerfacecolor='skyblue', markersize=4)
    plt.plot( columns.DAY_OF_YEAR, columns.QUANTILE_MAX_TEMP_FIFTEEN_DAYS, data=data_2plot, color='purple', linewidth=1)
    plt.plot( columns.DAY_OF_YEAR, columns.QUANTILE_MIN_TEMP_FIFTEEN_DAYS, data=data_2plot, color='green', linewidth=1)
    plt.legend()
    plt.xlabel(columns.DAY_OF_YEAR)
    plt.ylabel(columns.MAX_TEMP)
    plt.grid(True, linestyle='--', alpha = 0.5)
    plt.show()